# Combustion Model and Ignition Delay Demo

Written by A. Mark Payne and Alon Grinberg Dana for presentation at ICCK 2017

## User input:

In [ ]:
Fuel = 'CCO'
Equivalence_Ratio = 1.0

Temperature = 1500.0  # (K)
Pressure = 1.0        # (atm)
simtime = 2           # (ms)
topSA = 10            # number of top sensitive reactions and thermo to display

rmgpy_path = '../rmg.py' # Change to your rmg.py path


from IPython.display import display, Image
from rmgpy.molecule import Molecule
Fuel_Molecule = Molecule().fromSMILES(Fuel)
print("The fuel molecule is:");display(Fuel_Molecule)

## RMG's input file:

In [ ]:
Fuel_Molecule = Molecule().fromSMILES(Fuel)
nC = int(Fuel_Molecule.getNumAtoms('C'))
nH = int(Fuel_Molecule.getNumAtoms('H'))
nO = int(Fuel_Molecule.getNumAtoms('O'))

A = str(Equivalence_Ratio/(nC+(nH/4.0)-(nO/2.0)))

Input_file = '''
# Data sources
database(
    thermoLibraries = ['BurkeH2O2','primaryThermoLibrary','thermo_DFT_CCSDTF12_BAC','DFT_QCI_thermo','FFCM1(-)','JetSurF2.0'],
    reactionLibraries = ['BurkeH2O2inN2','FFCM1(-)','JetSurF2.0'],
    seedMechanisms = [],
    kineticsDepositories = ['training'],
    kineticsFamilies = 'default',
    kineticsEstimator = 'rate rules',
)

# List of species
species(
    label='fuel',
    reactive=True,
    structure=SMILES('''+"'"+Fuel+"'"+'''),
)

species(
    label='O2',
    reactive=True,
    structure=SMILES('[O][O]'),
)

species(
    label='N2',
    reactive=False,
    structure=SMILES('N#N'),
)

species(
    label='OH',
    reactive=True,
    structure=SMILES('[OH]'),
)

# Reaction system
simpleReactor(
    temperature=('''+str(Temperature)+''','K'),
    pressure=('''+str(Pressure)+''','atm'),
    initialMoleFractions={
        'fuel': '''+A+''',
        'O2': 1,
        'N2': 3.76,
    },
    terminationTime=(0.001,'s'),
    sensitivity=['OH'],
    sensitivityThreshold=0.01,
)

simulator(
    atol=1e-16,
    rtol=1e-8,
    sens_atol=1e-6,
    sens_rtol=1e-4,
)

model(
    toleranceKeepInEdge=0,
    toleranceMoveToCore=0.05,
    toleranceInterruptSimulation=0.05,
    maximumEdgeSpecies=300000
)

#pressureDependence(
#        method='modified strong collision',
#        maximumGrainSize=(0.5,'kcal/mol'),
#        minimumNumberOfGrains=250,
#        temperatures=(298,2500,'K',10),
#        pressures=(0.5,3,'bar',5),
#        interpolation=('Chebyshev', 6, 4),
#        maximumAtoms=16,
#)

options(
    units='si',
    generateOutputHTML=True,
    generatePlots=False,
    saveEdgeSpecies=False,
    saveSimulationProfiles=True,
    saveRestartPeriod=None,
)

generatedSpeciesConstraints(
    allowed=['input species','seed mechanisms','reaction libraries'],
    maximumCarbonAtoms=5,
    maximumOxygenAtoms=2,
    maximumNitrogenAtoms=0,
    maximumSiliconAtoms=0,
    maximumSulfurAtoms=0,
    maximumHeavyAtoms=6,
    maximumRadicalElectrons=2,
    allowSingletO2=False,
)


'''

import os
if not os.path.exists('RMG'):
    os.mkdir('RMG')
os.system('rm -r RMG/*')
with open('RMG/Demo.py','w') as RMG_Input_File:
    RMG_Input_File.write(Input_file)
print("Created RMG input file")

## Run RMG:

In [ ]:
os.system('python {0} RMG/Demo.py'.format(rmgpy_path))         

print("RMG Simulation Completed. Summary of log file:\n")
RMG_log = open('RMG/RMG.log','r').readlines()
lines = [x for x in RMG_log[-13:-1] if x != "\n"]
for line in lines: print(line)

## Run the generated model (using RMG's Cantera functions):

In [ ]:
from rmgpy.chemkin import *
from rmgpy.tools.canteraModel import *
from rmgpy.species import Species
import time

path = "RMG/chemkin/"
speciesList, reactionList = loadChemkinFile(path+'chem_annotated.inp',
                                            path+'species_dictionary.txt',
                                            path+'tran.dat')

nC = int(Fuel_Molecule.getNumAtoms('C'))
nH = int(Fuel_Molecule.getNumAtoms('H'))
nO = int(Fuel_Molecule.getNumAtoms('O'))
phi = Equivalence_Ratio
FuelStoich = phi/(nC+(nH/4.0)-(nO/2.0))

Fuel_Species=Species().fromSMILES(Fuel)
O2_Species=Species().fromSMILES('[O][O]')
N2_Species=Species().fromSMILES('N#N')
OH_Species=Species().fromSMILES('[OH]')
species_dict = getRMGSpeciesFromUserSpecies([Fuel_Species,O2_Species,N2_Species,OH_Species], speciesList)

reactorTypeList = ['IdealGasReactor']
reactionTimeList = ([simtime], 'ms')

molFracList=[{species_dict[Fuel_Species]: FuelStoich,
              species_dict[O2_Species]: 1,
              species_dict[N2_Species]: 3.76}]
Tlist = ([Temperature],'K')
Plist = ([Pressure],'atm')

job = Cantera(speciesList=speciesList, reactionList=reactionList, outputDirectory='')
job.loadChemkinModel(path+'chem_annotated.inp',transportFile=path+'tran.dat')
job.generateConditions(reactorTypeList, reactionTimeList, molFracList, Tlist, Plist)

alldata = job.simulate()
print("Done.")

## Plot:

In [ ]:
############### Settings ###############
fsize = (8,4) # Change to make the figure fit on your screen
########################################

import matplotlib.pyplot as plt
import pandas as pd
from rmgpy.tools import plot as rmg_plot
from operator import itemgetter
from rmgpy.tools.sensitivity import runSensitivity
%matplotlib notebook

times = alldata[0][0].data
pressures = alldata[0][1][1].data

dpdt = (pressures[1:] - pressures[:-1]) / (times[1:] - times[:-1])
idi = next(i for i,d in enumerate(dpdt) if d==max(dpdt))
ign_delay_time = times[idi]

for spc in xrange(len(alldata[0][1][:])):
    if alldata[0][1][spc].label == str(species_dict[Fuel_Species]):
        Fuel_idx = spc
    if alldata[0][1][spc].label == str(species_dict[OH_Species]):
        OH_idx = spc

for i in range(len(alldata[0][1][Fuel_idx].data)):
    if alldata[0][1][Fuel_idx].data[i]<0.001:
        Fuel_Depletion_Time = times[i]
        break

files = os.listdir('RMG/solver')

sensitivity_file = str(filter(lambda x: ('sensitivity' in x) and ('.csv' in x),files)[0])
SA_time, SA_data = rmg_plot.parseCSVData('RMG/solver/'+sensitivity_file)

time_error = 1

for i in range(len(SA_time.data)):
    if abs(SA_time.data[i]-ign_delay_time)<time_error:
        ign_delay_idx = i
        time_error = abs(SA_time.data[i]-ign_delay_time)

Gidx = 0
for i in xrange(len(SA_data[:])):
    if "G" in SA_data[i].label:
        if not Gidx:
            Gidx = i
        SA_data[i].label = SA_data[i].label.split('G')[1][1:-1]
    else:
        SA_data[i].label = SA_data[i].label.split(' ')[1]

rank1 = []
for n in xrange(Gidx):
    rank1.append(abs(SA_data[n].data[ign_delay_idx]))               # list of max SA range for each rxn
num1 = np.linspace(0,len(rank1)-1,len(rank1))        # will be used to get the order of reactions by rank1
num1 = zip(rank1,num1)
num1 = sorted(num1, key=itemgetter(0),reverse=True)
SA_k_data = []
SA_k_label = []
for i in xrange(min(topSA, Gidx)):
    SA_k_data.append(SA_data[int(num1[i][1])].data[ign_delay_idx])     # make sorted lists size topSA of SA values and rxns labels
    SA_k_label.append(SA_data[int(num1[i][1])].label)
        
rank2 = []
for n in xrange(len(SA_data)-Gidx):
    rank2.append(abs(SA_data[n+Gidx].data[ign_delay_idx]))     # list of max SA range for each rxn
num2 = np.linspace(0,len(rank2)-1,len(rank2))        # will be used to get the order of reactions by rank1
num2 = zip(rank2,num2)
num2 = sorted(num2, key=itemgetter(0),reverse=True)
SA_G_data = []
SA_G_label = []
for i in xrange(min(topSA, len(SA_data)-Gidx)):
    SA_G_data.append(SA_data[int(num2[i][1])+Gidx].data[ign_delay_idx])     # make sorted lists size topSA of SA values and rxns labels
    SA_G_label.append(SA_data[int(num2[i][1])+Gidx].label)
        
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['figure.autolayout'] = True

plt.style.use('ggplot')
plt.style.use('seaborn-pastel')

fig = plt.figure(figsize=fsize)

plt.subplot(1,2,1)
plt.plot(alldata[0][0].data*1000, alldata[0][1][Fuel_idx].data,'-o')
plt.xlabel('Time (ms)')
plt.ylabel('$Y_{Fuel}$')
plt.title('Fuel profile')
plt.xlim([0,2000*ign_delay_time])

plt.subplot(1,2,2)
plt.plot(alldata[0][0].data*1000, alldata[0][1][OH_idx].data,'-o')
plt.xlabel('Time (ms)')
plt.ylabel('$Y_{OH}$')
plt.title('OH profile')
plt.xlim([0,2000*ign_delay_time])
plt.arrow(0, alldata[0][1][OH_idx].data[idi], ign_delay_time*1000, 0, width=0.0001, head_width=0.0005, head_length=0.001, length_includes_head=True, color='r', shape='full')
plt.annotate(r'$Ignition Delay: \tau_{ign}$', xy=(0,0), xytext=(0, alldata[0][1][OH_idx].data[idi]+0.0005), fontsize=10);


fig = plt.figure(figsize=fsize)
plt.barh(np.arange(min(topSA, Gidx)), SA_k_data, 1/1.5, color="blue")
plt.gca().invert_yaxis()
plt.xlabel(r'Sensitivity: $\frac{\partial\:\ln{[OH]}}{\partial\:\ln{k}}$');
plt.rcParams.update({'axes.labelsize': 20})
plt.yticks(np.arange(min(topSA, Gidx)),SA_k_label)
plt.title("[OH] sensitivity to kinetics")

fig = plt.figure(figsize=fsize)
plt.barh(np.arange(min(topSA, len(SA_data)-Gidx)), SA_G_data, 1/1.5, color="blue")
plt.gca().invert_yaxis()
plt.xlabel(r'Sensitivity: $\frac{\partial\:\ln{[OH]}}{\partial\:G_i}$ $[mol/kcal]$');
plt.rcParams.update({'axes.labelsize': 20})
plt.yticks(np.arange(min(topSA, len(SA_data)-Gidx)),SA_G_label)
plt.title("[OH] sensitivity to thermo")

plt.show()

In [ ]:
gas = ct.Solution('RMG/cantera/chem.cti')
comp = str(species_dict[Fuel_Species])+":"+str(FuelStoich)+","+str(species_dict[O2_Species])+":1,"+str(species_dict[N2_Species])+":3.76"
gas.TPX = Temperature, Pressure, comp
reactor = ct.IdealGasConstPressureReactor(gas)
network = ct.ReactorNet([reactor])
network.advance(ign_delay_time)
ROP_C = ct.ReactionPathDiagram(gas, 'C')

from PIL import Image as PILimg
ROP1 = plt.subplot(1,1,1)
dot_file = 'RMG/cantera/rxnpathC.dot'
img_file = 'RMG/cantera/rxnpathC.png'
ROP_C.title = 'Reaction path diagram following C'
ROP_C.threshold = 0.01
ROP_C.label_threshold = 0.01
ROP_C.show_details = True
ROP_C.write_dot(dot_file)                                              # write dot file
os.system('dot {0} -Tpng -o{1} -Gdpi=300'.format(dot_file, img_file))   # write png file
fullpath = os.getcwd() + '/' + img_file
Image(fullpath)